## Adding new particles to a ParticleSet during runtime


There are use-cases, where it is important to be able to add particles 'on-the-fly', during the runtime of a Parcels simulation.

Unfortuantely, Parcels does not (yet) support adding new particles _within_ a `Kernel`. A workaround is to temporarily leave the `execution()` call to add particles via the `ParticleSet.add()` method, before continuing with `execution()`.

See the example below, where we add 'mass' to a particle each timestep, based on a probablistic condition, and then split the particle once its 'mass' is larger than 5


In [ ]:
GrowingParticle = parcels.Particle.add_variable(
    [
        parcels.Variable("mass", initial=0),
        parcels.Variable("splittime", initial=-1),
        parcels.Variable("splitmass", initial=0),
    ]
)


def GrowParticles(particles, fieldset):
    import random

    # 25% chance per timestep for particle to grow
    if random.random() < 0.25:
        particles.mass += 1.0
    growing_particles = (particles.mass >= 5.0) and (particles.splittime < 0)
    particles[growing_particles].splittime = particles.time
    particles[growing_particles].splitmass = particles.mass / 2.0
    particles[growing_particles].mass = particles.mass / 2.0


pset = parcels.ParticleSet(fieldset=fieldset, pclass=GrowingParticle, lon=0, lat=0, time=fieldset.time_interval.left)
outfile = parcels.ParticleFile("growingparticles.zarr", outputdt=np.timedelta64(1,'h'))



In [ ]:
for t in range(40):
    pset.execute(
        GrowParticles, runtime=np.timedelta64(1,'h'), dt=np.timedelta64(1,'h'), output_file=outfile, verbose_progress=False
    )
    for p in pset:
        if p.splittime > 0:
            pset.add(
                parcels.ParticleSet(
                    fieldset=fieldset,
                    pclass=GrowingParticle,
                    lon=0,
                    lat=0,
                    time=p.splittime,
                    mass=p.splitmass,
                )
            )
            p.splittime = -1  # reset splittime

INFO: Output files are stored in /Users/Gebruiker/Documents/GitHub/parcels/docs/examples_v3/growingparticles.zarr


TypeError: datetime64 values must have a unit specified

In [ ]:
np.dtype(np.uint64)

dtype('uint64')

The 'trick' is that we place the `pset.execute()` call in a for-loop, so that we leave the Kernel-loop and can add Particles to the ParticleSet.

Indeed, if we plot the mass of particles as a function of time, we see that new particles are created every time a particle reaches a mass of 5.


In [ ]:
ds = xr.open_zarr("growingparticles.zarr")
plt.plot(ds.time.values[:].astype("timedelta64[s]").T, ds.mass.T)
plt.grid()
plt.xlabel("Time")
plt.ylabel("Particle 'mass'")
plt.show()